# ERCOT Real-Time Energy Market Intelligence: GNN Modeling

**Research**: Real-time energy market intelligence and anomaly detection for the Texas ERCOT grid.

This notebook builds on the EDA and implements:
1. **GNN-based demand forecasting** (1h and 24h horizons)
2. **GNN-based anomaly detection** (grid stress, forecast busts)
3. **Baseline comparison** with XGBoost and Isolation Forest

In [ ]:
# Install dependencies (run once)
# !pip install torch pandas numpy scikit-learn xgboost requests matplotlib seaborn

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
from sklearn.ensemble import IsolationForest
import xgboost as xgb

from models.data_pipeline import (
    fetch_grid_load,
    fetch_weather,
    fetch_forecast,
    engineer_features,
    prepare_gnn_dataset,
)
from models.gnn_forecaster import train_gnn_forecaster, predict_gnn, ERCOTGNNForecaster
from models.gnn_anomaly import train_gnn_ae, detect_anomalies_gnn, GNNAutoEncoder

EIA_API_KEY = "2lXNIsctfqwa8zfH03rDca3IXcLnjgfN5dwUnsM4"   # Replace with your EIA API key
plt.style.use('seaborn-v0_8')
print("Environment ready.")

## 1. Load Data (Aligned with EDA Pipeline)

In [ ]:
print("Fetching ERCOT grid load...")
df_grid = fetch_grid_load(EIA_API_KEY)

start = df_grid.index.min().strftime('%Y-%m-%d')
end = (df_grid.index.max() - pd.Timedelta(days=5)).strftime('%Y-%m-%d')
print(f"Fetching weather for {start} to {end}...")
df_weather = fetch_weather(start=start, end=end)

df_raw = df_grid.join(df_weather, how='inner')
df_forecast = fetch_forecast(EIA_API_KEY)
if not df_forecast.empty:
    df_raw = df_raw.join(df_forecast, how='inner')

df_model = engineer_features(df_raw)
print(f"Data ready: {len(df_model)} rows")

## 2. Prepare GNN Dataset (Sliding Windows)

In [ ]:
SEQ_LEN = 24
HORIZON = 1
TRAIN_RATIO = 0.85

X_train, y_train, X_test, y_test, feature_names, adj = prepare_gnn_dataset(
    df_model,
    target_col='load_mw',
    seq_len=SEQ_LEN,
    horizon=HORIZON,
    train_ratio=TRAIN_RATIO,
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Features: {feature_names}")
print(f"Adjacency (correlation):\n{np.round(adj, 2)}")

## 3. Baseline: XGBoost (from EDA)

In [ ]:
features = ['hour', 'day_of_week', 'is_weekend', 'temperature_2m', 'load_lag_1h', 'load_lag_24h']
features = [f for f in features if f in df_model.columns]

split = int(len(df_model) * TRAIN_RATIO)
train_df = df_model.iloc[:split]
test_df = df_model.iloc[split:]

X_tr, y_tr = train_df[features], train_df['load_mw']
X_te, y_te = test_df[features], test_df['load_mw']

xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, early_stopping_rounds=30)
xgb_model.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False)

xgb_pred = xgb_model.predict(X_te)
xgb_mape = mean_absolute_percentage_error(y_te, xgb_pred)
xgb_mae = mean_absolute_error(y_te, xgb_pred)
print(f"XGBoost MAPE: {xgb_mape:.2%}, MAE: {xgb_mae:.0f} MW")

## 4. GNN Forecaster

In [ ]:
gnn_model, gnn_losses, scale_params = train_gnn_forecaster(
    X_train, y_train, adj,
    epochs=20,
    batch_size=128,
    lr=1e-3,
)

gnn_pred = predict_gnn(gnn_model, X_test, scale_params=scale_params)
gnn_mape = mean_absolute_percentage_error(y_test, gnn_pred)
gnn_mae = mean_absolute_error(y_test, gnn_pred)
print(f"\nGNN MAPE: {gnn_mape:.2%}, MAE: {gnn_mae:.0f} MW")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(gnn_losses, label='GNN Train Loss')
axes[0].set_title('GNN Training')
axes[0].legend()

n_show = min(168, len(y_test))
axes[1].plot(y_test[:n_show], label='Actual', linewidth=2)
axes[1].plot(gnn_pred[:n_show], label='GNN', linestyle='--')
axes[1].plot(xgb_pred[:n_show], label='XGBoost', linestyle='--', alpha=0.8)
axes[1].set_title('Forecast Comparison (1 week)')
axes[1].set_ylabel('Load (MW)')
axes[1].legend()
plt.tight_layout()
plt.show()

## 5. Anomaly Detection: Isolation Forest (Baseline) vs GNN

In [ ]:
anomaly_features = ['load_mw', 'temperature_2m']
if 'grid_stress' in df_model.columns:
    anomaly_features.append('grid_stress')
anomaly_features = [f for f in anomaly_features if f in df_model.columns]

iso = IsolationForest(contamination=0.02, random_state=42)
df_model['iso_anomaly'] = iso.fit_predict(df_model[anomaly_features])
iso_anomalies = (df_model['iso_anomaly'] == -1).sum()
print(f"Isolation Forest: {iso_anomalies} anomalies ({100*iso_anomalies/len(df_model):.1f}%)")

In [ ]:
ae_model, ae_losses = train_gnn_ae(X_train, adj, epochs=20, batch_size=64)

scores, mask, thresh = detect_anomalies_gnn(ae_model, X_train, threshold_percentile=98)
gnn_anomalies_train = mask.sum()

scores_test, mask_test, _ = detect_anomalies_gnn(ae_model, X_test, threshold_percentile=98)
gnn_anomalies_test = mask_test.sum()
print(f"GNN AE: {gnn_anomalies_train} train anomalies, {gnn_anomalies_test} test anomalies")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
n_plot = min(500, len(df_model))
ax.plot(df_model.index[:n_plot], df_model['load_mw'][:n_plot], label='Load', alpha=0.8)
iso_idx = df_model.index[df_model['iso_anomaly'] == -1]
iso_plot = iso_idx[iso_idx.isin(df_model.index[:n_plot])]
ax.scatter(iso_plot, df_model.loc[iso_plot, 'load_mw'], c='red', s=50, label='IF Anomaly', zorder=5)
ax.set_title('ERCOT Load with Isolation Forest Anomalies')
ax.legend()
plt.tight_layout()
plt.show()

## 6. Summary & Next Steps

In [ ]:
print("="*50)
print("ERCOT Modeling Summary")
print("="*50)
print(f"Demand Forecasting:")
print(f"  XGBoost MAPE: {xgb_mape:.2%}")
print(f"  GNN MAPE:     {gnn_mape:.2%}")
print(f"\nAnomaly Detection:")
print(f"  Isolation Forest: {iso_anomalies} events")
print(f"  GNN AutoEncoder:  {gnn_anomalies_test} test events")
print("="*50)
print("\nNext steps:")
print("  - Add zone-level data (EIA/ERCOT) for spatial GNN")
print("  - Integrate with ingest-grid-service for real-time")
print("  - Deploy dashboard with Streamlit/Dash")